In [21]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports

from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time
from GEMS_TCO import kernels_reparam_space_time_gpu_smooth15_011126 as kernels_reparam_space_time15
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, BaseLogger

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

Load daily data applying max-min ordering

In [36]:
space: List[str] = ['8', '8']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[-3,2]      
lon_range_input=[121, 131] 

df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,

lat_range=lat_range_input,   
lon_range=lon_range_input
  
)

In [37]:
daily_aggregated_tensors_dw = [] 
daily_hourly_maps_dw = []      

daily_aggregated_tensors_vecc = [] 
daily_hourly_maps_vecc = []   


for day_index in range(31):
    hour_start_index = day_index * 8
    
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_tensors_dw.append( day_aggregated_tensor )
    daily_hourly_maps_dw.append( day_hourly_map )

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= ord_mm,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_tensors_vecc.append( day_aggregated_tensor )
    daily_hourly_maps_vecc.append( day_hourly_map )
print(daily_aggregated_tensors_vecc[0].shape)
#print(daily_hourly_maps[0])
nn = daily_aggregated_tensors_vecc[0].shape[0]

torch.Size([2400, 4])


In [38]:
v=0.5
mm_cond_number= 8
nheads = 300
patience, factor = 5, 0.5


In [39]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Global L-BFGS Settings
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 7      # 10 to 20  
LBFGS_HISTORY_SIZE = 100   
LBFGS_MAX_EVAL = 100       # line search from 50 to 80
       
DELTA_LAT, DELTA_LON = 0.044, 0.063 
LAT_COL, LON_COL, VAL_COL, TIME_COL = 0, 1, 2, 3

days_list = [2]

# --- 2. Run optimization loop over pre-loaded data ---

for day_idx in days_list:  # 0-based

    print(f'\n{"="*40}')
    print(f'--- Starting Processing for Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'{"="*40}')

    # Assuming data access is correct
    daily_hourly_map_dw = daily_hourly_maps_dw[day_idx]
    daily_aggregated_tensor_dw = daily_aggregated_tensors_dw[day_idx]

    daily_hourly_map_vecc = daily_hourly_maps_vecc[day_idx]
    daily_aggregated_tensor_vecc = daily_aggregated_tensors_vecc[day_idx]

    if isinstance(daily_aggregated_tensor_vecc, torch.Tensor):
        daily_aggregated_tensor_vecc = daily_aggregated_tensor_vecc.to(DEVICE)


    init_sigmasq   = 13.059
    init_range_lat = 0.2
    init_range_lon = 0.25
    init_advec_lat = 0.0218
    init_range_time = 1.5
    init_advec_lon = -0.1689
    init_nugget    = 0.247

    #init_sigmasq   = 13.059
    #init_range_lat = 0.154 
    #init_range_lon = 0.195
    #init_advec_lat = 1e-8
    #init_range_time = 1e-8
    #init_advec_lon = 1e-8
    #init_nugget    = 0.247
    
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    # Create Initial Parameters (Float64, Requires Grad)
    initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                    np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

    params_list = [
        torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
        for val in initial_vals
    ]

    # --- 💥 Instantiate the GPU Batched Class ---
    # NOTE: Ensure fit_vecchia_lbfgs is the NEW class we defined
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map_vecc,
            aggregated_data = daily_aggregated_tensor_vecc,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )


    # --- 💥 Set L-BFGS Optimizer ---
    optimizer_vecc = model_instance.set_optimizer(
                params_list,     
                lr=LBFGS_LR,            
                max_iter=LBFGS_MAX_EVAL,        
                history_size=LBFGS_HISTORY_SIZE 
            )

    print(f"\n--- Starting Phase 2: Vecchia Optimization (Day {day_idx+1}) ---")
    start_time = time.time()
    
    # --- 💥 Call the Batched Fit Method ---
    # REMOVED: model_instance.matern_cov_aniso_STABLE_log_reparam
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer_vecc,
            # covariance_function argument is GONE
            max_steps=LBFGS_MAX_STEPS, 
            grad_tol=1e-7
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Vecchia Optimization finished in {epoch_time:.2f}s. Results: {out}")


Using device: cpu

--- Starting Processing for Day 3 (2024-07-3) ---

--- Starting Phase 2: Vecchia Optimization (Day 3) ---
Pre-computing Batched Tensors (Optimized CPU -> GPU)... Moving to GPU... Done. Heads: 2400, Tails: 0
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/7 / Loss: 1.728763 ---
  Param 0: Value=3.0867, Grad=7.747824415505863e-07
  Param 1: Value=0.5337, Grad=-2.3967925078551713e-06
  Param 2: Value=0.6701, Grad=4.947814352083669e-07
  Param 3: Value=-2.9728, Grad=-5.108258685528827e-07
  Param 4: Value=-0.0846, Grad=-1.1362351099712675e-05
  Param 5: Value=-0.2214, Grad=1.8460732387981516e-05
  Param 6: Value=1.2588, Grad=-8.027742477381565e-07
  Max Abs Grad: 1.846073e-05
------------------------------
--- Step 2/7 / Loss: 1.702054 ---
  Param 0: Value=3.0867, Grad=7.747824415505863e-07
  Param 1: Value=0.5337, Grad=-2.3967925078551713e-06
  Param 2: Value=0.6701, Grad=4.947814352083669e-07
  Param 3: Value=-2.9728, Grad=-5.108258685528827e-07
  Param 4

smooth 1.5

[8,8]
Final Interpretable Params: {'sigma_sq': 9.240691423790201, 'range_lon': 0.3232025361081519, 'range_lat': 0.2229692202976492, 'range_time': 1.4334804801466532, 'advec_lat': -0.08371538150117766, 'advec_lon': -0.22236605389882413, 'nugget': 6.634389909390918}
Vecchia Optimization finished in 14.02s. Results: [3.353092817765237, 1.1294761054967013, 0.7424908742456052, -2.9791629896526106, -0.08371538150117766, -0.22236605389882413, 1.8922667132009265, 1.7033197365112285]

[4,4]

Final Interpretable Params: {'sigma_sq': 9.104167862283989, 'range_lon': 0.26679639097180424, 'range_lat': 0.20742336147940024, 'range_time': 1.3812360812352726, 'advec_lat': -0.08644136965136057, 'advec_lon': -0.24167669613362816, 'nugget': 5.702427355679447}
Vecchia Optimization finished in 31.70s. Results: [3.5300018076384267, 1.3212694921243484, 0.5034477153572818, -3.2884966028475247, -0.08644136965136057, -0.24167669613362816, 1.740891936064917, 1.5851911682625786]


[1,1]

Final Interpretable Params: {'sigma_sq': 10.617270486144939, 'range_lon': 0.06627990183692334, 'range_lat': 0.06351684149909419, 'range_time': 0.5787318488493682, 'advec_lat': -0.0598335002733807, 'advec_lon': -0.22092296412947493, 'nugget': 1.2096283468791464}
Vecchia Optimization finished in 131.44s. Results: [5.076350533869923, 2.7138685674610312, 0.08516324065802365, -4.333905061416483, -0.0598335002733807, -0.22092296412947493, 0.19031316108846735, 1.1584158143987155]



smooth 0.5

[8.8]

Final Interpretable Params: {'sigma_sq': 12.84485573197888, 'range_lon': 0.5864029765044968, 'range_lat': 0.4194542477471602, 'range_time': 2.5925156087775454, 'advec_lat': -0.08464679304749435, 'advec_lon': -0.22144415035398135, 'nugget': 3.52127978238093}
Vecchia Optimization finished in 10.28s. Results: [3.086691451573345, 0.5337480525281425, 0.670105540535555, -2.972753468750143, -0.08464679304749435, -0.22144415035398135, 1.2588244980728351, 1.7020535816242846]

[4,4]
Final Interpretable Params: {'sigma_sq': 12.467028186492152, 'range_lon': 0.5809020516546307, 'range_lat': 0.4524420293278466, 'range_time': 3.0070961599839587, 'advec_lat': -0.08374944565455199, 'advec_lon': -0.23474917835848347, 'nugget': 3.4995797299125564}
Vecchia Optimization finished in 21.20s. Results: [3.0662605363831807, 0.543173122134782, 0.4998450277941496, -3.2882960086817197, -0.08374944565455199, -0.23474917835848347, 1.2526428841176867, 1.5835886096403093]
[1,1]
Final Interpretable Params: {'sigma_sq': 13.922837258857426, 'range_lon': 0.21171175344987925, 'range_lat': 0.20329476040381528, 'range_time': 1.4204296829867085, 'advec_lat': -0.06244026435314434, 'advec_lon': -0.22859075725937567, 'nugget': 2.2568863046773878e-07}
Vecchia Optimization finished in 153.70s. Results: [4.186060043219909, 1.5525295830160022, 0.0811374967896415, -3.8069780050191144, -0.06244026435314434, -0.22859075725937567, -15.304109528925483, 1.149011142993127]

구분,해상도,SigmaSq (σ2),Range Lon,Range Lat,Range Time,Nugget (τ2),Advec (Lon)
Smooth 0.5,"[8,8] (Sparse)",12.84,0.586,0.419,2.59,3.52,-0.221
(Rough),"[4,4] (Mid)",12.46,0.581,0.452,3.00,3.50,-0.234
,"[1,1] (Orig)",13.92,0.211,0.203,1.42,~0.00,-0.228
,,,,,,,
Smooth 1.5,"[8,8] (Sparse)",9.24,0.323,0.223,1.43,6.63,-0.222
(Smooth),"[4,4] (Mid)",9.10,0.267,0.207,1.38,5.70,-0.241
,"[1,1] (Orig)",10.62,0.066,0.064,0.58,1.21,-0.221

2. 민감도 분석 (Sensitivity Analysis)

① 데이터 밀집도에 따른 Range 파라미터의 왜곡 (Aliasing Effect)

두 모델 모두 데이터가 희소할수록(8,8) Range(상관 거리)를 실제보다 훨씬 길게 추정하고 있습니다.

현상: [8,8]에서 [1,1]로 갈수록 Range가 급격히 줄어듭니다. (Smooth 0.5는 Lon 기준 약 1/3토막, Smooth 1.5는 약 1/5토막)

원인: 희소한 데이터(Sparse)는 중간의 자잘한 변동(High frequency wiggle)을 보지 못합니다. 모델은 멀리 떨어진 두 점을 부드럽게 연결하려고 하므로, "상관관계가 멀리까지 지속된다"고 착각하게 됩니다.

결론: [8,8]이나 [4,4] 같은 희소 데이터로 학습한 모델은 실제 물리 현상보다 과대평가된 Range를 내놓으므로, Kriging(보간) 시 지나치게 부드러운 예측을 할 위험이 있습니다.

② Nugget(노이즈)의 변화와 "Roughness"

가장 결정적인 차이는 [1,1] 데이터에서의 Nugget 값입니다.

Smooth 0.5: Nugget이 0에 수렴합니다. (2.25e-07)

Smooth 1.5: Nugget이 여전히 1.21로 유의미하게 존재합니다.

분석:

원본 데이터는 매우 거친(Rough) 특성을 가지고 있습니다. (미분 불가능한 뾰족한 변동 등)

Smooth 0.5 (Exponential) 커널은 뾰족한(Non-differentiable) 특성을 허용하므로, 데이터의 거친 변동을 **"신호(Signal)"**로 받아들여 Nugget(노이즈)을 0으로 잡았습니다.

Smooth 1.5 (Matérn 3/2) 커널은 부드러운 곡선을 가정합니다. 데이터의 뾰족한 변동을 커널이 따라갈 수 없으니, 이를 **"노이즈(Nugget)"**로 밀어내버리거나, Range를 극단적으로 줄여서(0.06) 억지로 따라가려 합니다.



3. 눈에 띄는 다른 점 (Key Observations)

① Advection(이류) 파라미터의 놀라운 강건성 (Robustness)

가장 인상적인 점은 데이터가 희소하든 조밀하든, 커널이 거칠든 부드럽든 Advection 값은 거의 일정하다는 것입니다.

Advec Lon: 약 -0.22 ~ -0.24 사이에서 고정됨.

의미: 구름이나 오존의 이동 같은 **거시적인 흐름(Macro trend)**은 데이터를 듬성듬성 뽑아도([8,8]) 매우 정확하게 잡아낼 수 있다는 뜻입니다. 이는 물리적 해석이 아주 잘 되고 있음을 방증합니다.

② Smooth 1.5의 "Range 축소" 현상

보통 Smooth 1.5를 쓰면 Smooth 0.5보다 Range가 길게 나오는 경우가 많은데(동일한 ϕ일 때), 여기서는 반대로 Smooth 1.5의 Range([1,1] 기준 0.066)가 Smooth 0.5(0.211)보다 훨씬 짧습니다.

이것은 데이터가 Smooth 1.5로 설명하기 힘들 만큼 거칠기 때문입니다.

Smooth 1.5 모델이 거친 데이터를 설명하기 위해 "상관 거리가 아주 짧아서 금방금방 변하는 것이다"라고 억지로 피팅된 결과로 보입니다.

요약하자면:

데이터가 조밀해질수록 모델은 "아, 생각보다 상관성이 빨리 떨어지는구나(Range 감소)"라고 배웁니다.

이 데이터는 본질적으로 거칠어서(Rough), 부드러운 커널(1.5)보다는 뾰족한 커널(0.5)이 훨씬 더 잘 맞습니다.

1. 보통 Smooth 1.5를 쓰면 Smooth 0.5보다 Range가 길게 나오는 경우가 많은데(동일한 ϕ일 때), 여기서는 반대로 Smooth 1.5의 Range([1,1] 기준 0.066)가 Smooth 0.5(0.211)보다 훨씬 짧습니다.

이것은 데이터가 Smooth 1.5로 설명하기 힘들 만큼 거칠기 때문입니다.

2. semivariogram plot.  

1-1. smooth model tends to exaggerate nugget. see the plot and see if nugget values make sense. 

2-2. see the semivariogram near zero and see if it is linear (0.5)

3. likelihood 비교,  ratio of expected periodogram and empirical epriodograms. 

4. spectrum slope at high frequency (스무스하면 급격히 떨어지지)